# Generate xMelts liquid Endmembers 

A notebook to generate spud files for xMelts standard state models

Required system packages and initialization

In [ ]:
import os,sys
import pandas as pd
import numpy as np
import sympy as sym
import time
import warnings
sym.init_printing()

Required ENKI packages

In [ ]:
from thermocodegen.coder import coder

### let's set up some directory names for clarity

In [ ]:
HOME_DIR = os.path.abspath('../../')
SPUD_DIR = HOME_DIR+'/endmembers'
try:
    os.makedirs(SPUD_DIR)
except:
    pass

Set a reference string for this Notebook

In [ ]:
reference = 'fo_sio2/notebooks/Generate_xmelts_endmembers.ipynb'

In [ ]:
df = pd.read_csv('data/xMelts.csv')
df.columns

In [ ]:
# pull out column names required for just standard state model
cols = ['Phase', 'Formula',  'H_TrPr', 'S_TrPr', 'V_TrlPr', 
           'T_fus', 'S_fus', 'Cpl',
           'k0', 'k1', 'k2', 'k3', 'v1', 'v2', 'v3', 'v4']

Rename some columns for use with spud files

In [ ]:
df = df[cols]
df = df.rename(index=str,columns={'Phase':'name', 'Formula':'formula'})

restrict to just Forsterite and Quartz4

In [ ]:
names = ['Forsterite', 'Quartz4']
df = df[df.name.isin(names)]
df.set_index('name')

## Generate Sympy symbols and Coder parameters from the column headers

To construct our Gibbs free energy we need to create Sympy symbols and units for the parameters
* H_TrPr: Enthalpy at reference $T$ and $P$ in $J$
* S_TrPr: Entropy at reference T and P in $J/K$
* V_TrPr: Volume at reference T and P in $J/bar$
* k0-k3:  Coefficients in Berman heat capacity model with units [ 'J/K-m', 'J-K^(1/2)-m', 'J-K/m', 'J-K^2' ]
* v1-v4:  Coefficients in Berman Equation of state (V) with units [ '1/bar', '1/bar^2' , '1/K', '1/K^2']
* Tg:  Cp transition Temperature
* Trl: liquid Reference Temperature



In [ ]:
param_strings = df.columns[2:]
param_strings = param_strings.insert(0,'Trl')
units = [ 'K','J', 'J/K', 'J/bar-m','K','J/K', 'J/K', 'J/K-m','J-K^(1/2)-m','J-K/m' ,'J-K^2',
        '1/bar', '1/bar^2', '1/K',  '1/K^2']
print('parameter\tunit')
for t in list(zip(param_strings, units)):
    print('{}\t\t {}'.format(t[0],t[1]))

We now construct coder parameters and sympy symbols from the parameter names and pass them to the local dictionary for use in other sympy expressions

In [ ]:
params = coder.set_coder_params(param_strings, units)
symbol_dict = coder.get_symbol_dict_from_params(params)

# load local dictionary
locals().update(symbol_dict)


## initialize the coder model (Gibbs model)

In [ ]:
model= coder.StdStateModel.from_type()

Set up reference symbols (Trl already defined)

In [ ]:
T = model.get_symbol_for_t()
P = model.get_symbol_for_p()
Tr = model.get_symbol_for_tr()
Pr = model.get_symbol_for_pr()

## Standard State Potentials

### Define model Potentials for the Standard State Potentials
An expression for the Gibbs free energy, $G(T,P)$ 

#### (1) $C_P$ integrals
The isobaric heat capacity terms parameterized as: 

$$
    C_P = k_0 + k_1 / T^{1/2} + k_2 / T^2 + k_3 / T^3,
$$

and in addition the reference condition third law entropy, $S_{Tr,Pr}$, and enthalpy of formation from the
elements, $\Delta H_{Tr,Pr}$, constitute additional parameters:

In [ ]:
# Solid Heat Capacity
Cps = k0+k1/sym.sqrt(T)+k2/T**2+k3/T**3

In [ ]:
# Enthalpy and Entropy

In [ ]:
H = H_TrPr + sym.integrate(Cps,(T,Tr,T_fus))   + sym.integrate(Cpl,  (T,T_fus,T)) + T_fus*S_fus

In [ ]:
S = S_TrPr + sym.integrate(Cps/T,(T,Tr,T_fus)) + sym.integrate(Cpl/T,(T,T_fus,T)) + S_fus

In [ ]:
GPr = H  - T*S
GPr

#### (2) $V$ (EOS) integrals
Next, define a volume-explicit equation of state applicable over the whole of temperature and pressure space


Liquid volume

In [ ]:
half = sym.Rational(1,2)
Vf = V_TrlPr + (T - Trl)*(v1 + half*v3*(P - Pr)) + (P - Pr)*(v2 + half*v4*(P - Pr))

In [ ]:
GPrToP = sym.integrate(Vf,(P,Pr,P))

#### Define standard state G and parameters

In [ ]:
G = GPr + GPrToP
G
G.simplify()

### Generate a coder model and write to spud files

In [ ]:
model.add_potential_to_model('G',G, params)

Exam settable values for the model dictionary

In [ ]:
values_dict = model.get_values()    
values_dict

### Write out spud files for all Berman standard state endmembers

In [ ]:
for i, row in df.iterrows():
    values_dict = row.to_dict()
    # clean up names to make them coder compliant
    values_dict['name'] ='{}_xmelts'.format(values_dict['name'].title())
    values_dict['name'] = values_dict['name'].replace('-','_')
    values_dict['Trl'] = 1673.15
    print(values_dict['name'])
    values_dict['reference']=reference
    model.set_values(values_dict)
    model.to_xml(path=SPUD_DIR)